In [1]:
import torch.nn as nn
import sys
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = "/content/drive/My Drive/project/"
sys.path.append(path + "URP")
sys.path.append(path + "URP/learning")
sys.path.append(path + "URP/data")
sys.path.append(path + "URP/models")
sys.path.append(path + "URP/forgetting")

In [11]:
from data_utils import show_random, AddNoise, remove_random, remove_class #, separate_data
from data import get_dataset, dummy_clusters, dummy_half_doughnuts
from models_utils import *
import models
from learn import *
from forget import *

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
cnn = {"filters_percentage": 1, "n_channels":3,"num_classes": 50, "dropout": True, "batch_norm": True}
cnn_model = get_model("cnn", **cnn)
print(cnn_model.__class__.__name__)

CNN


In [7]:
  checkpoint = torch.load(f"/content/drive/My Drive/project/URP/checkpoints/{cnn_model.__class__.__name__}_{60}.pth.tar")
  cnn_model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [12]:
cifar10_train, cifar10_val, cifar10_test = get_dataset("cifar10", batch_size=32, cuda=0,
                  train_size=10000, val_size=1000, test_size=500,
                  test_batch_size=10)

Files already downloaded and verified
Dataset sizes: 	 train: 10000 	 val: 1000 	 test: 500
Batch size: 	 32


In [13]:
forget, retain = remove_class(cifar10_train, [1])

In [15]:
neg_gradient(cnn_model, loss='ce', optimizer='adam', epochs=40, device=device, dataset=forget, lossfn=None, train_loader=forget,
    scheduler=None, weight_decay=0.0, lr=0.01, momentum=0.9)

Optimizer: 	  adam

Loss function:
CrossEntropyLoss()
[Logging in CNN_training]
[0] train metrics:{"loss": 0.0011069272877648473, "error": 0.0}
Epoch number: 0 :
 Epoch Time: 0.68 sec
[1] train metrics:{"loss": 0.00015292831812985241, "error": 0.0}
Epoch number: 1 :
 Epoch Time: 0.66 sec
[2] train metrics:{"loss": 0.00014620687579736114, "error": 0.0}
Epoch number: 2 :
 Epoch Time: 0.68 sec
[3] train metrics:{"loss": 3.642704541562125e-05, "error": 0.0}
Epoch number: 3 :
 Epoch Time: 0.66 sec
[4] train metrics:{"loss": 3.4848766517825425e-05, "error": 0.0}
Epoch number: 4 :
 Epoch Time: 0.66 sec
[5] train metrics:{"loss": 2.515636515454389e-05, "error": 0.0}
Epoch number: 5 :
 Epoch Time: 0.67 sec
[6] train metrics:{"loss": 1.8381886548013426e-05, "error": 0.0}
Epoch number: 6 :
 Epoch Time: 0.64 sec
[7] train metrics:{"loss": 9.373241482535377e-06, "error": 0.0}
Epoch number: 7 :
 Epoch Time: 0.65 sec
[8] train metrics:{"loss": 1.5120691614356474e-06, "error": 0.0}
Epoch number: 8 :
 

In [17]:
!ls ./checkpoints/

CNN_29.pth.tar	CNN_39.pth.tar


In [20]:
test(model=cnn_model, loss='ce', lossfn=None, optimizer='adam', device=device, dataset='cifar10', test_loader=retain, at_epoch=39)

Optimizer: 	  adam

Loss function:
CrossEntropyLoss()
TESTING
[0] test metrics:{"loss": 5.251279830932617, "error": 0.8620689660310745}
FINISHED TESTING


In [21]:
test(model=cnn_model, loss='ce', lossfn=None, optimizer='adam', device=device, dataset='cifar10', test_loader=forget, at_epoch=39)

Optimizer: 	  adam

Loss function:
CrossEntropyLoss()
TESTING
[0] test metrics:{"loss": 41.143951416015625, "error": 1.0}
FINISHED TESTING
